--------------------------------------------------------------------------------------

* Team member names: Kevin Wong, Lukas Fullner
* Team member IDs: A17280855

--------------------------------------------------------------------------------------

# Mini-project 4, DSC 170, Winter 2025

# Suitability Modeling

This project will focus on __suitability analysis__ with raster data. Your tasks will be both conceptual-level and technical. 

There are three parts to this assignment.

1. At the conceptual level, you will define a suitability model of your choice, for an area of your choice (preferably San Diego, because we already have worked with some local data). Consult the lectures on map combination, and also see https://pro.arcgis.com/en/pro-app/latest/help/analysis/spatial-analyst/suitability-modeler/the-general-suitability-modeling-workflow.htm for a brief description of what a suitability model is. For example, you may be looking for best areas for community gardens. Such areas are often selected from underutilized land in residential land uses, with good soils, good drainage, accessible (not steep slope), etc. So you would be looking for areas with a specific type of land use/land cover, with an appropriate range of values of slope, etc. You may build additional criteria based on a range of precipitaiton values, whether the area is affected by wildfires, or has low levels of soil erosion, etc. Feel free to use the imagery layers we explored or mentioned during raster-focused lectures. Several cells in lecture notebooks contained URLs to imagery layer collections available through AGOL - but feel free to find more. Also, feel free to download additional raster layers from elsewhere (an example in lecture demonstrates how to do this from the USGS image repository, and also how to get Sentinel-2 from ESA), publish them on ArcGIS Enterprise, and use in your model. 

You can use any __two__ of the <i>map combination</i> techniques discussed during lectures. You should identify the ones you use and discuss any uncertainty issues associated with these specific map combination models. 

As the outcome of this part, you will need to: a) describe the suitability model you want to develop; b) identify the raster data layers you will use; and c) describe two of the map combination techniques you will use to derive the two suitablity maps, and their pros and cons.

2. The second part will involve implementing your suitability model using arcgis raster functions. Many of these functions are new and experimental! Examples of what works are in the lecture notebooks. Be creative!    

3. The third part will be a brief write-up comparing the two output rasters generated for the suitability models using the two map combination techniques. 

The notebook should include documentation of the steps, as usually.



**Due Date: 03/01/2025 11:59PM (Pacific Time)**

**Total Possible Points: 30 Pts**

## Task 1: Formulate a suitability or risk model (5 points)

Your text here (about 100 words)

To help identify possible locations for public parks in San Diego, we could look at a few options, including Population Density and Demographics (Areas with high population density and little park access), neighborhoods with more seniors and children, and more. We could also look at park distribution (areas beyond walking distance of parks), and check for availability of land in case we could repurpose public land. Some physical characteristics that we would typically look for would be flat lane (slope), decent soil quality, with good drainage and accessibility. Additionally, we could think about accessibility, with proximity to homes, public transportation, bike lanes, and walkability of the neighborhoods surrounding the area. 

Some map combination techniques we could use include a weighted linear combination to balance the social need of a community versus the physical suitability of the land. We could also use Boolean Overlay to help determine areas that aren't risk to fires or floods, which would help narrow down areas that we would like to use.

## Task 2: Implement the model (20 points)

In [1]:
# Imports, etc.
import arcgis
import arcgiskey
from arcgis.gis import GIS
from arcgis.geometry import *
from arcgis.raster.functions import *
from ipywidgets import *
import graphviz
from IPython.display import display

gis = GIS(username = arcgiskey.USERNAME, password = arcgiskey.PASSWORD)

In [115]:
# List imagery layers to be used in your model. 
# This cell should contian layer definitions.
usa_landcover_gap = ImageryLayer(
    "https://landscape3.arcgis.com/arcgis/rest/services/USA_Landcover_GAP/ImageServer",
    gis,
)
usa_soil_erosion = ImageryLayer(
    "https://landscape11.arcgis.com/arcgis/rest/services/USA_Soils_Erosion_Class/ImageServer",
    gis,
)
usa_elevation = ImageryLayer(
    "https://elevation.arcgis.com/arcgis/rest/services/WorldElevation/Terrain/ImageServer",
    gis,
)
usa_federal_lands = ImageryLayer(
    "https://landscape10.arcgis.com/arcgis/rest/services/USA_Federal_Lands/ImageServer",
    gis,
)
usa_flood_hazard = ImageryLayer(
    "https://landscape11.arcgis.com/arcgis/rest/services/USA_Flood_Hazard_Areas/ImageServer",
    gis,
)


In [116]:
# Derive the area of interest (AOI) and its geometry and extent. 
# The smaller the area the better (so that you don't run into raster size limitations)

# set the geometry and boundary extent for San Diego county specifically
counties = gis.content.search("USA Counties generalized", "Feature Layer", outside_org=True)[4]
county_layer = counties.layers[0]
county = county_layer.query("NAME = 'San Diego County'", return_geomtry = True)
san_diego_geometry = county.features[0].geometry
san_diego_geometry['spatialRef'] = san_diego_geometry['spatialReference']
san_diego_extent = Geometry(san_diego_geometry).geoextent

# create the extent as a tuple
tup = (
    ("xmin", san_diego_extent[0]),
    ("ymin", san_diego_extent[1]),
    ("xmax", san_diego_extent[2]),
    ("ymax", san_diego_extent[3]),
)
sd_ex = dict(tup)

# 2d: to be correcter, let's also add CRS:
crs = "{'latestwkid':3857, 'wkid':102100}"
sd_ex.update({"spatialReference": {"latestwkid": 3857, "wkid": 102100}})

In [117]:
# Set the extent if needed
# San Diego Land Cover Gap
usa_landcover_gap.extent = sd_ex
sd_lcg = clip(usa_landcover_gap, san_diego_geometry)

# USA Soil Erosion
usa_soil_erosion.extent = sd_ex
sd_se = clip(raster = usa_soil_erosion, geometry = san_diego_geometry)

# USA Elevation
usa_elevation.extent = sd_ex
sd_elev = clip(raster = usa_elevation, geometry = san_diego_geometry)

# USA Federal Lands
usa_federal_lands.extent = sd_ex
sd_fl = clip(raster = usa_federal_lands, geometry = san_diego_geometry)

# USA Flood Hazard
usa_flood_hazard.extent = sd_ex
sd_fh = clip(raster = usa_flood_hazard, geometry = san_diego_geometry)

In [136]:
# LOOK AT 'VALUE' for remap and 'NVC_CLASS' for how I classified
sd_lcg.attribute_table()['features']

[{'attributes': {'OID': 0,
   'VALUE': 1,
   'COUNT': 348934,
   'RED': 0,
   'GREEN': 0.51765,
   'BLUE': 0.4902,
   'CL': '01',
   'NVC_CLASS': 'Forest & Woodland',
   'SC': '01.A',
   'NVC_SUBCL': 'Tropical Moist Forest',
   'FRM': '01.A.03',
   'NVC_FORM': 'Tropical Flooded & Swamp Forest',
   'DIV': 'D002',
   'NVC_DIV': 'Neotropical Flooded & Swamp Forest',
   'MACRO_CD': 'M001',
   'NVC_MACRO': 'Caribbean & Central American Flooded & Swamp Forest',
   'LEVEL3': 9238,
   'ECOLSYS_LU': 'South Florida Bayhead Swamp',
   'NVCMES': '01.A.3.D002M001E001'}},
 {'attributes': {'OID': 1,
   'VALUE': 2,
   'COUNT': 1140832,
   'RED': 0,
   'GREEN': 0.51765,
   'BLUE': 0.4902,
   'CL': '01',
   'NVC_CLASS': 'Forest & Woodland',
   'SC': '01.A',
   'NVC_SUBCL': 'Tropical Moist Forest',
   'FRM': '01.A.03',
   'NVC_FORM': 'Tropical Flooded & Swamp Forest',
   'DIV': 'D002',
   'NVC_DIV': 'Neotropical Flooded & Swamp Forest',
   'MACRO_CD': 'M001',
   'NVC_MACRO': 'Caribbean & Central American

In [132]:
# Remap and Normalize the Outputs of each layer
# Remap the Land Cover Gap layer
# For the Land Cover Gaps, we will remap the values to the following classes:
# Shrubland & Grassland, Agricultural Vegatation, Introduced & Semi Natural Vegatation, Recently Disturbed or Mofidified, and Developed & Other Human Use to Green
# Forest & Woodland, Semi_deset, Polar & High Montane Vegetation Vegatation, Aquatic Vegetation, and Nonvascular * Sparse Vacular Rock Vegetation & Opem Water to Red
clrmap2 = [[1, 230, 0, 0], [0, 38, 115, 0]]
sd_lcg_norm = remap(
    raster=sd_lcg,
    input_ranges=[1.0,289.0, 290.0,459.0, 460.0,499.0, 500.0,507.0, 508.0,513.0,
                   514.0,554.0, 555.0,557.0, 558.0,563.0, 564.0,576.0,
                     577.0,579.0, 580.0,585.0],
    output_values=[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0],
)
sd_lcg_binary = colormap(sd_lcg_norm, colormap=clrmap2)


In [134]:
map2 = gis.map("San Diego County")
map2.content.add(sd_lcg_binary)
map2

Map(center=[3899047.926834046, -12998800.702954108], extent={'xmin': -13068598.023681488, 'ymin': 3816090.6311…

In [103]:
sd_lcg.attribute_table()['features']

[{'attributes': {'OID': 0,
   'VALUE': 1,
   'COUNT': 348934,
   'RED': 0,
   'GREEN': 0.51765,
   'BLUE': 0.4902,
   'CL': '01',
   'NVC_CLASS': 'Forest & Woodland',
   'SC': '01.A',
   'NVC_SUBCL': 'Tropical Moist Forest',
   'FRM': '01.A.03',
   'NVC_FORM': 'Tropical Flooded & Swamp Forest',
   'DIV': 'D002',
   'NVC_DIV': 'Neotropical Flooded & Swamp Forest',
   'MACRO_CD': 'M001',
   'NVC_MACRO': 'Caribbean & Central American Flooded & Swamp Forest',
   'LEVEL3': 9238,
   'ECOLSYS_LU': 'South Florida Bayhead Swamp',
   'NVCMES': '01.A.3.D002M001E001'}},
 {'attributes': {'OID': 1,
   'VALUE': 2,
   'COUNT': 1140832,
   'RED': 0,
   'GREEN': 0.51765,
   'BLUE': 0.4902,
   'CL': '01',
   'NVC_CLASS': 'Forest & Woodland',
   'SC': '01.A',
   'NVC_SUBCL': 'Tropical Moist Forest',
   'FRM': '01.A.03',
   'NVC_FORM': 'Tropical Flooded & Swamp Forest',
   'DIV': 'D002',
   'NVC_DIV': 'Neotropical Flooded & Swamp Forest',
   'MACRO_CD': 'M001',
   'NVC_MACRO': 'Caribbean & Central American

In [98]:
unique_pairs = {}
for feature in at['features']:
    attributes = feature['attributes']
    cl = attributes['CL']
    nvc_class = attributes['NVC_CLASS']
    if cl not in unique_pairs:
        unique_pairs[cl] = nvc_class

unique_pairs

{'01': 'Forest & Woodland',
 '02': 'Shrubland & Grassland',
 '03': 'Semi-Desert',
 '04': 'Polar & High Montane Vegetation',
 '05': 'Aquatic Vegetation',
 '06': 'Nonvascular & Sparse Vascular Rock Vegetation',
 '07': 'Agricultural Vegetation',
 '09': 'Introduced & Semi Natural Vegetation',
 '10': 'Recently Disturbed or Modified',
 '11': 'Open Water',
 '08': 'Developed & Other Human Use'}

In [82]:
sd_lcg.properties.maxValues

[584]

In [80]:
map1 = gis.map("San Diego County, CA")
map1.content.add(sd_lcg)
map1

Map(center=[3899047.926834046, -12998800.702954108], extent={'xmin': -13068598.023681488, 'ymin': 3816090.6311…

### Name the two map combination techniques you will use to combine the data and describe their pros and cons

Your text here 

In [ ]:
# Prepare your input layers for map combination: clip to AOI, remap/normalize, visualize the layers. 


In [ ]:
# Generate a composite raster layer for your first map combination technique


In [ ]:
# Generate a composite raster layer for your second map combination technique


## Task 3: Compare the results (5 points)
... and describe how different combination techniques resulted in different outputs (or not.) 

Your text here


In [ ]:
## Timekeeping
# Please let us know how much time you spent on this project, in hours: 
# (we will only examine distributions and won't look at individual responses)

assignment_timespent = 0